In [ ]:
%matplotlib inline

In [ ]:
import gc
import pathlib
import sys

In [ ]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import PoissonRegressor

In [ ]:
src_dir = "../../src"

In [ ]:
sys.path.append(src_dir)

In [ ]:
from package.constants import *

In [ ]:
train = pd.read_parquet(train_path)

In [ ]:
is_train = train["date"] >= "2011-02-07"

In [ ]:
train = train[is_train]

In [ ]:
grouped = train.groupby([pd.Grouper(key="date", freq="7D"), "item_id"])
y = grouped[target].sum()
y = y.unstack()

n_samples, _ = y.shape
X = np.arange(n_samples)
X = X.reshape(-1, 1)

In [ ]:
coef = pd.Series(1.0, name="coef", index=y.columns)

In [ ]:
for col in y:
    model = PoissonRegressor()

    notnull = y[col].notnull()

    model.fit(X[notnull], y.loc[notnull, col])

    coef[col] += model.coef_

In [ ]:
coef = coef.astype("float32")

In [ ]:
coef.plot(kind="hist", figsize=(16, 9), grid=True, bins=100)

In [ ]:
joblib.dump(coef, coef_path)